In [1]:
import cv2 #opencv
import mediapipe as mp
import math
import csv
import datetime
import pandas as pd
import os
from mediapipe_estimate import estimate
pd.options.mode.chained_assignment = None 

In [2]:
test_dat=data=pd.read_csv('golfDB_front_orig.csv',index_col='id')
test_dat=test_dat[['youtube_id','events_2','events_5','events_7']]
test_dat['events_5']=test_dat['events_5']-test_dat['events_2']+15
test_dat['events_7']=test_dat['events_7']-test_dat['events_2']+15

test_dat=test_dat.drop(['events_2'],axis=1)
test_dat

,youtube_id,events_5,events_7
id,,,
8,-M5SITXMA2Y,45,53
10,gOBVh7fzyZo,39,47
14,vN3Uc_EhnnY,41,49
19,iW323nsTGtU,49,57
21,xD6KDqPF9cc,39,45
...,...,...,...
1370,g90r9cs1tTw,37,43
1372,eNlBbMI-MhQ,49,59
1374,6K1FD0sOwoI,54,63


In [3]:
name_list=list('output_video_'+pd.unique(test_dat.youtube_id))
all=len(name_list)
correct_contact=0
correct_top=0
correct_address=0
for fl in name_list:
    try:
        data=pd.read_csv('results_both_wrists/variables_'+fl+'.csv')
        
        #consider only time between backswing and finish
        halfway_back_ind=data['right_wrist_x'].idxmin()
        halfway_front_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
        middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_front_ind)]

        #find moment of ball contact as the lowest wrist point on y
        contact_frame=middle_data[middle_data['right_wrist_y']==middle_data['right_wrist_y'].max()].index
        
        #isolate only backswing data
        back_data=data[(data.index<contact_frame.item())]

        #find moment of top of backswing as the highest wrist point on y
        top_backswing_frame=back_data[back_data['right_wrist_y']==back_data['right_wrist_y'].min()].index

        #find moment before start of the swing as the lowest wrist point on y before going halfway back
        halfway_back_data=data[data['right_wrist_x'].index<halfway_back_ind]
        address_frame=halfway_back_data[halfway_back_data['right_wrist_y']==halfway_back_data['right_wrist_y'].max()].index
        
        #extract the gt from golfDB
        yt_id='_'.join(fl.split('_')[2::])
        gt_contact=test_dat[test_dat.youtube_id==yt_id].events_7.item()
        gt_top=test_dat[test_dat.youtube_id==yt_id].events_5.item()
        gt_address=15

        #calculte tolerance as per golfDB paper (fps=30)
        tolerance_hit=2*(gt_contact//30)
        tolerance_top=2*(gt_top//30)
        tolerance_address=2*(gt_address//30)



        if gt_contact-tolerance_hit<=contact_frame.item()<=tolerance_hit+gt_contact:
            correct_contact+=1
            print('correct hit esitmate',correct_contact,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_contact].timestamp.item(),', tolerance [frames]:',tolerance_hit, ', frame difference:', abs(contact_frame.item()-gt_contact))
        else:
            print('incorrect hit estimate',correct_contact,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),', gt [s]:', data[data.index==gt_contact].timestamp.item(),', tolerance [frames]:',tolerance_hit,', frame difference:', abs(contact_frame.item()-gt_contact))

        if gt_top-tolerance_top<=top_backswing_frame.item()<=tolerance_top+gt_top:
            correct_top+=1
            print('correct top esitmate',correct_top,' estimated [s]:',data[data.index==top_backswing_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_top].timestamp.item(),', tolerance [frames]:',tolerance_top, ', frame difference:', abs(top_backswing_frame.item()-gt_contact))
        else:
            print('icorrect top esitmate',correct_top,' estimated [s]:',data[data.index==top_backswing_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_top].timestamp.item(),', tolerance [frames]:',tolerance_top, ', frame difference:', abs(top_backswing_frame.item()-gt_contact))
      
        if gt_address-tolerance_address<=address_frame.item()<=tolerance_address+gt_address:
            correct_address+=1
            print('correct address esitmate',correct_address,' estimated [s]:',data[data.index==address_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_address].timestamp.item(),', tolerance [frames]:',tolerance_address, ', frame difference:', abs(address_frame.item()-gt_address))
        else:
            print('icorrect address esitmate',correct_address,' estimated [s]:',data[data.index==address_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_address].timestamp.item(),', tolerance [frames]:',tolerance_address, ', frame difference:', abs(address_frame.item()-gt_address))
        print('----------------------------------------------------------------------------------------------------')
    except Exception as e:
        print(e,fl)
        print('----------------------------------------------------------------------------------------------------')
        all=all-1
        
              
            
accuracy_contact=correct_contact/all
accuracy_top=correct_top/all
accuracy_address=correct_address/all

print(f'contatct estimation accuracy: {accuracy_contact}, top estimation accuracy: {accuracy_top}, address estimation accuracy: {accuracy_address}')

incorrect hit estimate 0  estimated [s]: 1.43 , gt [s]: 1.77 , tolerance [frames]: 2 , frame difference: 10
icorrect top esitmate 0  estimated [s]: 1.33 gt [s]: 1.5 , tolerance [frames]: 2 , frame difference: 13
icorrect address esitmate 0  estimated [s]: 0.7 gt [s]: 0.5 , tolerance [frames]: 0 , frame difference: 6
----------------------------------------------------------------------------------------------------
correct hit esitmate 1  estimated [s]: 1.53 gt [s]: 1.57 , tolerance [frames]: 2 , frame difference: 1
correct top esitmate 1  estimated [s]: 1.27 gt [s]: 1.3 , tolerance [frames]: 2 , frame difference: 9
icorrect address esitmate 0  estimated [s]: 0.0 gt [s]: 0.5 , tolerance [frames]: 0 , frame difference: 15
----------------------------------------------------------------------------------------------------
attempt to get argmin of an empty sequence output_video_vN3Uc_EhnnY
----------------------------------------------------------------------------------------------------

In [4]:
# data=pd.read_csv('results/variables_'+fl+'.csv')
# halfway_back_ind=data['right_wrist_x'].idxmin()
# data_back=data[data.index>halfway_back_ind]
# data['sum_wrists_hit']=data['right_wrist_y']+data.timestamp
# ev_6=data[data['sum_wrists_hit']==data['sum_wrists_hit'].max()].index